In [22]:
from pathlib import Path
import vtk
import numpy as np

In [23]:
def read_vtu(filepath):
    stepCompression_6vtu = vtk.vtkXMLUnstructuredGridReader()
    stepCompression_6vtu.SetFileName(str(filepath))
    stepCompression_6vtu.Update()
    output_compression = stepCompression_6vtu.GetOutput()
    return output_compression

filepath = Path('/home/simoneponcioni/Documents/01_PHD/03_Methods/HFE/99_TEMP/damage_maps/C0003091_02/Step-Compression_4.vtu')
output_compression = read_vtu(filepath)

In [35]:
def decomposition(JacobianArray):
    SphericalCompression = np.zeros(JacobianArray.shape[-1])
    IsovolumicDeformation = np.zeros(JacobianArray.shape[-1])
    for k in range(0, JacobianArray[0]):
        for j in range(0, JacobianArray[1]):
            for i in range(0, JacobianArray[2]):

                F_d = JacobianArray[k, j, i, :].reshape((3, 3))

                ## Unimodular decomposition of F
                J = np.linalg.det(F_d)
                SphericalCompression[k, j, i] = J

                if J > 0:
                    F_tilde = J ** (-1 / 3) * F_d
                    Norm_F_tilde = np.linalg.norm(F_tilde)
                else:
                    Norm_F_tilde = 0.0

                IsovolumicDeformation[k, j, i] = Norm_F_tilde
    return SphericalCompression, IsovolumicDeformation

def decomposition_simone(F):
    spherical_compression = np.zeros(F.shape[-1])
    isovolumic_deformation = np.zeros(F.shape[-1])
    for i in range(F.shape[-1]):
        spherical_compression[i] = np.linalg.det(F[:, :, i])
        if spherical_compression[i] > 0:
            F_tilde = spherical_compression[i] ** (-1 / 3) * F[:, :, i]
            isovolumic_deformation[i] = np.linalg.norm(F_tilde)
        else:
            isovolumic_deformation[i] = 0.0
    return spherical_compression, isovolumic_deformation


arrays = output_compression.GetCellData()
centroid_arrays = {arrays.GetArrayName(i): arrays.GetArray(i) for i in range(arrays.GetNumberOfArrays()) if 'Centroid' in arrays.GetArrayName(i)}

ones_array = np.ones(centroid_arrays['SDV_F11_Centroid'].GetNumberOfTuples())

F11 = centroid_arrays['SDV_F11_Centroid']
F12 = centroid_arrays['SDV_F12_Centroid']
F13 = centroid_arrays['SDV_F13_Centroid']
F21 = centroid_arrays['SDV_F21_Centroid']
F22 = centroid_arrays['SDV_F22_Centroid']
F23 = centroid_arrays['SDV_F23_Centroid']
F31 = centroid_arrays['SDV_F31_Centroid']
F32 = centroid_arrays['SDV_F32_Centroid']
F33 = centroid_arrays['SDV_F33_Centroid']
F = np.array([F11, F12, F13, F21, F22, F23, F31, F32, F33]).reshape(3, 3, -1)

spherical_compression, isovolumic_deformation = decomposition_simone(F)
# Create new arrays for spherical compression and isovolumic deformation
spherical_compression_array = vtk.vtkFloatArray()
spherical_compression_array.SetName("SphericalCompression")
isovolumic_deformation_array = vtk.vtkFloatArray()
isovolumic_deformation_array.SetName("IsovolumicDeformation")

# Set the values of the arrays
for i in range(len(spherical_compression)):
    spherical_compression_array.InsertNextValue(spherical_compression[i])
    isovolumic_deformation_array.InsertNextValue(isovolumic_deformation[i])

# Add the arrays to the output_compression vtk unstructured grid
output_compression.GetCellData().AddArray(spherical_compression_array)
output_compression.GetCellData().AddArray(isovolumic_deformation_array)

# Save the vtk unstructured grid with all the cell data
writer = vtk.vtkXMLUnstructuredGridWriter()
writer.SetFileName('/home/simoneponcioni/Documents/01_PHD/03_Methods/HFE/99_TEMP/damage_maps/C0003091_02/Step-Compression_4_with_data.vtu')
writer.SetInputData(output_compression)
writer.Write()


1